In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
DEVICE = torch.device("cuda:0")
# Moving to the gpu: net = Net().to(DEVICE)

In [127]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        encode = []
        relu = nn.ReLU(True)
        pool = nn.MaxPool2d(2)
        encode.append(nn.Sequential(nn.Conv2d(1, 32, 3, padding=1), relu, pool))
        encode.append(nn.Sequential(nn.Conv2d(32, 64, 3, padding=1), relu, pool))
        encode.append(nn.Sequential(nn.Conv2d(64, 128, 3, padding=1), relu, pool))
        encode.append(nn.Sequential(nn.Conv2d(128, 256, 3, padding=1), relu, pool))
        encode.append(nn.Sequential(nn.Conv2d(256, 256, 3, padding=1), relu))
        
        self.encoders = []
        self.encoders.append(nn.Sequential(*encode)) # for n-1
        self.encoders.append(nn.Sequential(*encode)) # for D(n-1)
        self.encoders.append(nn.Sequential(*encode)) # for n
        self.encoders.append(nn.Sequential(*encode)) # for D(n+1)
        self.encoders.append(nn.Sequential(*encode)) # for n+1
        
        decode = []
        decode.append(nn.Sequential(nn.Conv2d(256*5, 256, 3, padding = 1), relu))
        decode.append(nn.Sequential(nn.ConvTranspose2d(256*6, 128, 3, stride=2, padding=1, output_padding=1), relu))
        decode.append(nn.Sequential(nn.ConvTranspose2d(128*6, 64, 3, stride=2, padding=1, output_padding=1), relu))
        decode.append(nn.Sequential(nn.ConvTranspose2d(64*6, 32, 3, stride=2, padding=1, output_padding=1), relu))
        decode.append(nn.Sequential(nn.ConvTranspose2d(32*6, 1, 3, stride=2, padding=1, output_padding=1), relu))
        
        self.decoder = nn.Sequential(*decode)
        '''
        transform = []
        transform.append(nn.Sequential(nn.Conv3d(1, 32, 3), relu))
        transform.append(nn.Sequential(nn.Conv2d(32, 1, 3), relu))
        
        self.transformer = nn.Sequential(*transform)
        '''
    def encodedecode(self, x):
        skips = []
        add = []
        for i in range(0,5):
            skips.append(add)
            
        nextinput = x
        for i, encoder in enumerate(self.encoders):
            x = nextinput
            for layer in encoder:
                x = layer(x)
                skips[i].append(x)
        x = skips[-1][-1]
        for i, layer in enumerate(self.decoder):
            for j, encoder in enumerate(skips):
                if j == len(skips)-1 and i == 0: break
                x = torch.cat((x,encoder[-(i+1)]),1)
            x = layer(x)
        return x
        
    def forward(self, x):
        x = self.encodedecode(x)
        return x

network = autoencoder()
x = torch.randn(1,1280,720)
network(x.view(-1,1,1280,720)).size()


torch.Size([1, 256, 80, 45])
torch.Size([1, 128, 160, 90])
torch.Size([1, 64, 320, 180])
torch.Size([1, 32, 640, 360])
torch.Size([1, 1, 1280, 720])


torch.Size([1, 1, 1280, 720])

In [25]:
class testnet(nn.Module):
    def __init__(self):
        super().__init__()
        relu = nn.ReLU(True)
        self.t1  = nn.Sequential(nn.Conv3d(1, 32, 3, 3), relu)
        self.t2  = nn.Sequential(nn.Conv2d(32, 1, 3, 3), relu, nn.MaxPool2d((30,20)))

    def forward(self, x):
        x = self.t1(x)
        x = self.t2(x.view(1, 32, 426, 240))
        return x
testnet = testnet()

In [28]:
rand  = torch.randn(1, 2, 1280, 720)

testnet(rand.view(-1, 1, 2, 1280, 720)).size()

RuntimeError: Calculated padded input size per channel: (2 x 1280 x 720). Kernel size: (3 x 3 x 3). Kernel size can't be greater than actual input size